In [1]:
import pandas as pd
import numpy as np
import string
import pymorphy2
import nltk
import time

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, precision_recall_fscore_support

In [3]:
df_pos = pd.read_csv("data_pos.csv", sep=",", header = 0)
df_neg = pd.read_csv("data_neg.csv", sep=",", header = 0)
df_neu = pd.read_csv("data_neu.csv", sep=",", header = 0)

In [4]:
del df_pos['Unnamed: 0']
del df_neg['Unnamed: 0']
del df_neu['Unnamed: 0']

In [5]:
df_neg['sentiment'] = -1
df_neu['sentiment'] = 0
df_pos['sentiment'] = 1

In [6]:
df = pd.concat([df_neg, df_neu, df_pos], ignore_index=True)

dataframe со всеми рецензиями:

In [7]:
df.head()

,review,sentiment
0,Культовый трэш-хоррор Уэса Крейвена начинается...,-1
1,"Зашла на свой любимый сайт, полностью убежденн...",-1
2,"Помимо того, что они любят спасать мир по-свое...",-1
3,Решил посмотреть данный сериал из-за смеси ком...,-1
4,В своё время студия 'Мельница' подарила нам не...,-1


In [8]:
df.shape

(131669, 2)

In [9]:
data = df.copy()

In [10]:
data.head()

,review,sentiment
0,Культовый трэш-хоррор Уэса Крейвена начинается...,-1
1,"Зашла на свой любимый сайт, полностью убежденн...",-1
2,"Помимо того, что они любят спасать мир по-свое...",-1
3,Решил посмотреть данный сериал из-за смеси ком...,-1
4,В своё время студия 'Мельница' подарила нам не...,-1


#### Предобработка текста
Удаляем стоп-слова, знаки пунктуации и понижаем регистр

In [ ]:
nltk.download('stopwords')

In [ ]:
def preprocess_text(text, mystopwords = stopwords.words('russian')):
    text = " ".join([word for word in text.split() if (word not in mystopwords) and (word.count(word[0]) != len(word))])
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [ ]:
data['review'] = [preprocess_text(t) for t in data['review']]

In [ ]:
data.head()

Стэмминг:

In [ ]:
def stem_text(text):
    snowball = SnowballStemmer(language="russian")
    text = " ".join([snowball.stem(word) for word in text.split()])
    return text

In [ ]:
data['review'] = [stem_text(t) for t in data['review']]

In [ ]:
data.head()

Лемматизация:

In [ ]:
def lemm_text(text):
    morph = pymorphy2.MorphAnalyzer()
    text = " ".join([morph.parse(word)[0].normal_form for word in text.split()])
    return text

In [ ]:
data['review'] = [lemm_text(t) for t in data['review']]

In [ ]:
data.head()

Разобьем данные на тренировочную (70%) и тестовую (30%) выборки.

In [ ]:
x_train, x_test = train_test_split(data, test_size=0.3, random_state=1)

Векторизация текста

In [ ]:
count_vectorizer = CountVectorizer(analyzer="word", ngram_range=(1, 1))

In [ ]:
train_data_features = count_vectorizer.fit_transform(x_train['review'])

Кросс-валидация

In [ ]:
# svm = LinearSVC(dual=False)
mnb= MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
k_fold = KFold(5, shuffle=True, random_state=1)
res = cross_val_score(mnb, train_data_features, x_train['sentiment'], cv=k_fold)
print(res)
print(np.mean(res))

Вспомогательная функция вывода метрик

In [ ]:
def evaluate_prediction(predictions, target):
    print('accuracy %s' % accuracy_score(target, predictions))
    print('precision %s' % precision_recall_fscore_support(target, predictions)[0])
    print('recall %s' % precision_recall_fscore_support(target, predictions)[1])
    print('fscore %s' %precision_recall_fscore_support(target, predictions)[2])
    print('Precision =', precision_score(target, predictions, average='micro'))
    print('Recall =', recall_score(target, predictions, average='micro'))
    print('F1 =', f1_score(target, predictions, average='micro'))

Функция, предсказывающая значения

In [ ]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['review'])
    predictions = classifier.predict(data_features)
    target = data['sentiment']
    target = target.astype('int')
    evaluate_prediction(predictions, target)
    return predictions

Обучаем модель, предсказываем значения

In [ ]:
y_train = x_train['sentiment']
y_train = y_train.astype('int')

# svm = LinearSVC(dual=False)
mnb = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

start_time = time.time()
# svm.fit(train_data_features, y_train)
mnb.fit(train_data_features, y_train)
print("--- %s seconds(обучение) ---" % (time.time() - start_time))

start_time = time.time()
predictions = predict(count_vectorizer, mnb, x_test)
print("--- %s seconds(предсказание) ---" % (time.time() - start_time))

In [ ]:
y_test = x_test['sentiment']
conf_mat = confusion_matrix(y_test, predictions)
print("Матрица несоответствий:")
print(conf_mat)

Основные показатели набора данных

In [ ]:
def senten(data):
    k = 0
    for review in data['review']:
        k += review.count('.')
    return k / data.shape[0]


def words(data):
    k = 0
    for review in data['review']:
        k += len(review.split())
    print(k)
    return k / data.shape[0]


def uniq_words(data):
    words_dict = dict()
    for review in data['review']:
        for word in review.split():
            if word in words_dict:
                words_dict[word] = words_dict[word] + 1
            else:
                words_dict[word] = 1
    return len(words_dict)

In [ ]:
print("Количество уникальных слов =", uniq_words(df))
print("Среднее количество предложений в отзыве =", senten(df))
print("Среднее количество слов в отзыве =",words(df))